In [1]:
import iparser
import importlib

## Топ 10-15 профессий:

 

* Java разработчик/Software engineer
* Front-end разработчик/Web разработчик
* Продуктовый дизайнер
* Системный аналитик
* Архитектор систем
* Agile coach (под вопросом?)
* Data Scientist
* Data engineer
* Бизнес-аналитик
* Финансовый аналитик
* Кредитный аналитик
* Юрист (корпоративный, судебный)
* Маркетолог
 

 

In [107]:
professions = [
"Java разработчик ",
"Software engineer",
"Web разработчик",
"Front-end разработчик",
"Продуктовый дизайнер",
"Системный аналитик",
"Архитектор систем",
"Agile coach",
"Data Scientist",
"Data engineer",
"Бизнес-аналитик",
"Финансовый аналитик",
"Кредитный аналитик",
"Юрист корпоративный",
"Юрист судебный",
"Маркетолог",
]

In [108]:
professions

['Java разработчик ',
 'Software engineer',
 'Web разработчик',
 'Front-end разработчик',
 'Продуктовый дизайнер',
 'Системный аналитик',
 'Архитектор систем',
 'Agile coach',
 'Data Scientist',
 'Data engineer',
 'Бизнес-аналитик',
 'Финансовый аналитик',
 'Кредитный аналитик',
 'Юрист корпоративный',
 'Юрист судебный',
 'Маркетолог']

In [60]:
int(63.3123412341341234)

63

In [103]:
importlib.reload(iparser)
p = iparser.IndeedParser()
p.fit(keywords='Java разработчик', region="Moscow")
p.run()

https://ru.indeed.com/jobs?q=Java+разработчик&l=Moscow


In [98]:
import pandas as pd
vacs = pd.DataFrame(p.list_of_vacancy_links, columns = ['link'])

In [100]:
vacs.shape

(46, 1)

In [102]:
vacs.drop_duplicates().shape

(38, 1)

In [94]:
vacs = vacs.drop_duplicates()

In [95]:
vacs

,title,link
0,Data scientist,/rc/clk?jk=1f6b2bf33ffe4058&fccid=da4a9c4c078b...
1,Data Scientist в Погоду,/rc/clk?jk=782b9ed67f4b250f&fccid=aefdff46dd76...
2,Scientist,/rc/clk?jk=c77123b4008ded0b&fccid=e4af9ca5b6d3...
3,Data Scientist,/rc/clk?jk=7adf94f3f7c9bd5d&fccid=66578288116f...
4,Data Scientist (Predictive analytics),/rc/clk?jk=c394045d0b89059b&fccid=4b65c1e9ec07...
5,Data Scientist,/rc/clk?jk=379cee83c1c01640&fccid=854b734f22e2...
6,Data scientist,/rc/clk?jk=39ee6e84958bbb11&fccid=358674dc7181...
7,Data Scientist / Machine Learning Engineer,/rc/clk?jk=7e27d6329350a4e2&fccid=361e19bc9924...
8,Data scientist,/rc/clk?jk=2998c77a42813dc3&fccid=fa51d16a6da4...
9,Data Scientist,/rc/clk?jk=18789e8199117641&fccid=a63391bea190...


In [109]:
importlib.reload(iparser)
vacancy_dict = {}
for prof in tqdm(professions):
    print(prof)
    p = iparser.IndeedParser()
    p.fit(keywords=prof, region="Moscow")
    p.run()
    vacancy_dict[prof] = p.vacancy_texts
    

Java разработчик 
https://ru.indeed.com/jobs?q=Java+разработчик&l=Moscow


ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [112]:
import pickle
with open("vacs_dictNY.pkl", "wb") as f:
    pickle.dump(vacs, f)

In [114]:
with open("vacs_dict.pkl", "rb") as f:
    vacancy_dict = pickle.load(f)

In [117]:
len(vacancy_dict['Java разработчик '])

439

In [118]:
vacancy_dict['Java разработчик '][0]

'Команда самого технологичного каршеринга ждет смелого разработчика для работы над клиентским приложением и не только. Это возможность оказаться в проекте в самом начале бурного роста. Нужно уметь писать код. А еще искать решения, общаться и делать все быстро. Потом переделывать. Вам понравится.\nТребования:\nзнание Java и Kotlin;\nопыт разработки под Android от года, хорошее знание API Android начиная с 5.0;\nпонимание Аctivity/Fragment Lifecycle;\nзнание архитектур MVC, MVVM, MVP, понимание работы протокола HTTP, навыки работы с потокобезопасным кодом;\nзнание и понимание основных алгоритмов и структур данных;\nнавыки оптимизации приложений по потреблению памяти и по FPS.\nДля начала разговора расскажите о своем опыте и навыках.'

In [119]:
vacancy_dict['Java разработчик '][1]

'Обязанности\nРазработка депозитарной системы;\nРуководство территориально-распределенной командой разработки;\nАнализ требований и оценка трудозатрат;\nПроведение Code Review;\nРазвитие процессов CI/CD.\n\nТребования\nВысшее техническое образование;\nОпыт разработки на Java от 3-х лет;\nЗнание Java 8/Spring;\nЗнание JavaScript;\nОпыт работы с Docker;\nОпыт работы с Git;\nОпыт разработки микросервисов.\nЖелательно:\nОпыт работы от 2 лет на позиции Ведущего разработчика/Team Lead с управлением командой из 3-5 человек;\nОпыт разработки на Groovy;\nОпыт работы с Gradle;\nОпыт работы с Jasper Reports;\nОпыт работы с БД Oracle/разработки на PL/SQL;\nОпыт работы с Kubernetes.\n\nУсловия\nГрафик работы с 9:00 до 18:00, в пятницу с 9:00 до 16:45;\nОформление согласно ТК РФ;\nСоциальный пакет, включающий расширенную медицинскую страховку, ДМС, ММС; страхование от несчастных случаев;\nПрограммы обучения и развития для сотрудников: тренинги, изучение английского языка;\nНасыщенная корпоративная ж

In [120]:
vacancy_dict['Java разработчик '][2]

'Требования к кандидату\n\n\nЗнание MySQL — написание SQL-запросов.\n\n\nОчень приветствуется опыт работы с Laravel или с другими фреймворками.\n\n\nУмение писать и оптимизировать сложные запросы к БД.\n\n\nHTML, CSS, Java Script. Но при большом желании делать только backend, можем это обеспечить.\n\n\nLess и git — приветствуется опыт использования.\n\n\nАналитические способности, вдумчивость, логическое мышление, нацеленность на результат.\n\n\nБыстрая обучаемость, желание развиваться.\n\n\nОбязанности\n\n\nСоздание, развитие и поддержка веб-проектов.\n\n\nСоздание новых сервисов.\n\n\nПоддержка и развитие Личного кабинета клиентов.\n\n\nНаписание и оптимизация сложных запросов к БД.\n\n\nУчастие в разработке новых программных продуктов.\nВакансии компании\nСлужба персонала\nEmail: izdat@urait.ru\n\nТел.: +7 (495) 744-00-12, доб. 110, 111\nПривет! Мечтаешь развиваться в сфере образования? — Присоединяйся к команде образовательной платформы «Юрайт»\nМы ищем творческих и амбициозных сот

In [121]:
vacancy_dict['Java разработчик '][3]

'Москва, Россия\nОсновные проекты\nВ Бэлл Интегратор на направлении Банки ведутся проекты – Интернет банк для юридический лиц, Платформа поддержки развития бизнеса, Единая фронтальная система и Фабрика данных\nКоманды разработчиков и аналитиков Бэлл Интегратор принимают участие в разработке, доработке и модернизации этих платформ\nОбязанности\nУчастие в проекте по доработке и развитию системы дистанционного банковского обслуживания для корпоративных клиентов\nРеализация компонентов: адаптеры получения данных, компоненты сервера приложений, веб сервисы\nРазработка интерфейсов взаимодействия с системами и сервисами\nРазработка интерфейсов программного доступа к сервисам на ESB\nТребования\nВысшее образование\nJAVA – опыт разработки промышленных приложений от 3-х лет\nОпыт работы c Java Core, collections, multithreading\nЗнание библиотек: Spring, IoC, DI\nЗнание паттернов проектирования ПО\nЗнание JSON, XML, SQL\nОпыт работы с Maven, Gradle\nЗнание ORM Hibernate, понимание подходов code-f

In [122]:
vacancy_dict['Java разработчик '][4]

'Отдел инфраструктуры разработки ищет талантливого специалиста в команду сервиса управления исходным кодом для разработчиков Яндекса.\n\nЕжедневно через наш сервис проходит более 2000 изменений кода и более 1000 успешных ревью кода. Наш продукт упрощает и ускоряет все аспекты работы с огромной кодовой базой Яндекса: поиск и навигация, отслеживание изменений, ревью кода, покрытие кода тестами и многое другое доступно в нашем сервисе и развивается с молниеносной скоростью.\n\nПеред нашей командой стоит огромное количество важных задач, в которых нам нужна ваша помощь: поддержка новых систем контроля версий, поиск и представление документации по коду, профили разработчиков, интеграция с внутренними системами управления релизами, непрерывной интеграции, распределенной сборки и др. В нашем продукте есть возможность воплотить в жизнь самые смелые идеи, которые делают ежедневную работу с кодом быстрее и приятнее.\n\nЧто означает работа в нашей группе:\nразнообразие задач: от молниеносных прод

In [125]:
vaca = vacancy_dict['Java разработчик '][5].split("\n")

In [ ]:
vaca

In [130]:
vaca = [x.lower() for x in vaca]

In [137]:
for el in vaca:
    if el.strip()in ['навыки','требования', 'ожидания']:
        print(el, vaca.index(el))
        skillposnum = vaca.index(el)
    if el.strip()in ['условия','предлагаем',"предложение"]:
        termsposnum = vaca.index(el)
        print(el, vaca.index(el))
    if skillposnum < termsposnum:
        skillspart = vaca[skillposnum:termsposnum]

навыки 21
условия 24


In [138]:
skillspart

['навыки',
 'что мы ожидаем от кандидата:',
 'уверенные знания kotlin, java, android sdk;опыт разработки клиент-серверных приложений;опыт разработки мобильных приложений c применением принципов solid;понимание принципов ооп;опыт работы с шаблонами проектирования и умение их применять;знание основных алгоритмов и структур данных;умение писать поддерживаемый и тестируемый код;умение разбираться в чужом коде;опыт написания юнит-тестов;наличие завершенных проектов в google play.']

In [143]:
skillsss= [x for x in skillspart if x not in ['навыки','требования', 'ожидания', 'ожидание','ожидаем']]

In [148]:
skillslen = [len(x) for x in skillsss]
maxlenpos = skillslen.index(max(skillslen))
s = skillsss[maxlenpos]

In [150]:
s.split(";")

['уверенные знания kotlin, java, android sdk',
 'опыт разработки клиент-серверных приложений',
 'опыт разработки мобильных приложений c применением принципов solid',
 'понимание принципов ооп',
 'опыт работы с шаблонами проектирования и умение их применять',
 'знание основных алгоритмов и структур данных',
 'умение писать поддерживаемый и тестируемый код',
 'умение разбираться в чужом коде',
 'опыт написания юнит-тестов',
 'наличие завершенных проектов в google play.']

In [152]:
from parse_hh_data import download, parse
vacancy = download.vacancy("36070814")
resume = download.resume("d40ce6f80001a8c8380039ed1f5874726f5a6e")
resume = parse.resume(resume)